In [1]:
import numpy as np
from osgeo import ogr, gdal, osr
import rasterio
import json


import matplotlib.pyplot as plt
import pandas as pd
import os.path
import re

#### Resize DEM from 30 to 10m

In [2]:
# Reading Normal DEM data
area_dem = '/home/shrayank_mistry/Modules/project-mum-pune/dem_clipped.tif'
dem = rasterio.open(area_dem, count = 1)

dem = np.array(dem.read(1))
print(dem.shape)

(2721, 3610)


In [3]:
# Up-sampling DEM to 10m resolution
import rasterio
from rasterio.enums import Resampling

# whole-numbers indicate upscaling, fractions indicate downscaling
upscale_factor = 3


with rasterio.open('/home/shrayank_mistry/Modules/project-mum-pune/dem_clipped.tif') as dataset:

    # resample data to target shape
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * upscale_factor),
            int(dataset.width * upscale_factor)
        ),
        resampling=Resampling.bilinear
    )

    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )

In [4]:
dem = data[0]
print(dem.shape)

(8163, 10830)


In [5]:
# reading LULC data
area_cover = '/home/shrayank_mistry/Modules/project-mum-pune/mask.tif'
area = rasterio.open(area_cover, count = 1)

area = np.array(area.read(1))

In [6]:
dem_data, mask_data = dem, area

print(dem_data.shape, mask_data.shape)
width, height = 10500, 8163

dem_data, mask_data = dem_data[:height,:width], mask_data[:height, :width]
print(dem_data.shape, mask_data.shape)

dem_data, mask_data = dem_data.T, mask_data.T
print(dem_data.shape, mask_data.shape)

(8163, 10830) (8273, 10500)
(8163, 10500) (8163, 10500)
(10500, 8163) (10500, 8163)


#### AHP Mapping

In [7]:
ahp_map = {

'urban': 0.29,
'farms': 0.239,
'dense-forest': 0.207,
'water': 0.13,
'fallow': 0.067,
'sparse-forest': 0.049,
'barren-land': 0.019,
'unclassified':7,

}

class_map = {

    0: 'unclassified',
    1: 'water',
    2: 'dense-forest',
    3: 'sparse-forest',
    4: 'barren-land',
    5: 'urban',
    6: 'farms',
    7: 'fallow',

}

In [8]:
# Helper function to map from LULC to class-map to ahp values
def set_weights(c):
    c_str = class_map.get(c)
    return ahp_map[c_str]

In [9]:
set_weights_vtr = np.vectorize(set_weights)
mask_data = set_weights_vtr(mask_data)

In [10]:
np.unique(mask_data, return_counts = True)

KeyboardInterrupt: 

#### Start-End Co-ordinates

In [ ]:
# ----------------------------- source_shp -------------------------------- #

file = ogr.Open("/home/shrayank_mistry/Modules/project-mum-pune/route-files/source-point.shp")
source_shp = file.GetLayer(0)

feature = source_shp.GetFeature(0)
source_shp = feature.ExportToJson()

source_shp = json.loads(source_shp)
start_ext = source_shp['geometry']['coordinates']

# ----------------------------- destination_shp -------------------------------- #

file = ogr.Open("/home/shrayank_mistry/Modules/project-mum-pune/route-files/destination-point.shp")
destination_shp = file.GetLayer(0)

feature = destination_shp.GetFeature(0)
destination_shp = feature.ExportToJson()

destination_shp = json.loads(destination_shp)
end_ext = destination_shp['geometry']['coordinates']

In [ ]:
print(start_ext, end_ext) 

[300378.7763074944, 2104126.3099621385] [366123.3883685307, 2063699.8154670105]


#### Raster Co-ordinates (Left, Bottom, Right, Top)

In [ ]:
path = '/home/shrayank_mistry/Modules/project-mum-pune/raster.tif' 

data = rasterio.open(path)
print(data.bounds)

extent = data.bounds

left, bottom, right, top = extent[0], extent[1], extent[2], extent[3]
print(left, bottom, right, top)

In [ ]:
width = round(right - left)
height = round(top - bottom)

print("Height and Width of Raster")
print(height, width)

Height and Width of Raster
82730 105000


### Finding pixels of start-point and end-point from extents

In [ ]:
# [Do not run - already avaliable]
# start_pixel, end_pixel = [14424, 17699], [54850, 83443]
# start_pixel, end_pixel = [0, 0], [0, 0]
# start_flag, end_flag = True, True

# for i in range(height):
#     for j in range(width):

#         if (not start_flag) and (not end_flag):
#             break
        
#         # print(round(left + j), round(top - i))
#         if (start_flag and ((round(left + j)) == round(start_ext[0])) and ((round(top - i)) == round(start_ext[1]))):
#             start_pixel = [i, j]
#             start_flag = False

        
#         if (end_flag and ((round(left + j)) == round(end_ext[0])) and ((round(top - i)) == round(end_ext[1]))):
#             end_pixel = [i, j]
#             end_flag = False

In [ ]:
start_pixel, end_pixel = [14424, 17699], [54850, 83443]

In [ ]:
start_pixel = np.array(start_pixel)
start_pixel = start_pixel / 10

start_pixel = list(np.rint(start_pixel))
start_pixel = list(np.array(start_pixel, dtype = 'int'))

end_pixel = np.array(end_pixel)
end_pixel = end_pixel / 10

end_pixel = list(np.rint(end_pixel))
end_pixel = list(np.array(end_pixel, dtype = 'int'))

In [ ]:
print(start_pixel, end_pixel)
start_pixel, end_pixel = [1770, 1442], [8344, 5485]
print(start_pixel, end_pixel)

[1442, 1770] [5485, 8344]
[1770, 1442] [8344, 5485]


#### A* search algorithm

In [ ]:
def condition_check(start, end):
    if (start[0] == end[0]) and (start[1] == end[1]):
        return False
    return True

In [ ]:
import math
from queue import PriorityQueue

In [ ]:
mask_data = mask_data * 5000

In [ ]:
np.unique(mask_data, return_counts = True)

(array([   95.,   245.,   335.,   650.,  1035.,  1195.,  1450., 35000.]),
 array([19822048, 17097029, 28893954,  7807154,  3858436,  5940554,
         2291737,      588]))

In [ ]:
def c_anist_cost(i, j, x, y, mu = 10, wt = 2):

    mu_sqr = mu * mu
    h_diff = dem_data[i][j] - dem_data[x][y]
    h_sqr = h_diff * h_diff
    c_dv = (mask_data[i][j] + mask_data[x][y]) / 2
    cst = np.sqrt(mu_sqr + h_sqr) * (c_dv + math.atan(h_diff / mu) * wt) + acc_cost[i][j]

    return cst

In [ ]:
def get_neigh_cost(i, j):
    arr = []

    #(1) col - 1, row - 1
    if (j - 1 >= 0) and (i - 1 >= 0):
        g = c_anist_cost(i, j, i - 1, j - 1)
        # h = h_cost_euclidean(i - 1, j - 1)
        h = h_cost_manhattan(i - 1, j - 1)

        if (acc_cost[i - 1][j - 1] > (g + h)):
            parent[i - 1][j - 1] = i, j
            acc_cost[i - 1][j - 1] = (g + h)
        arr.append([acc_cost[i - 1][j - 1], [i - 1, j - 1], [i, j]])
    else:
        arr.append(math.inf)

    #(2) col, row - 1
    if (i - 1 >= 0):
        g = c_anist_cost(i, j, i - 1, j)
        # h = h_cost_euclidean(i - 1, j)
        h = h_cost_manhattan(i - 1, j - 1)

        if (acc_cost[i - 1][j] > (g + h)):
            parent[i - 1][j] = i, j
            acc_cost[i - 1][j] = (g + h)
        arr.append([acc_cost[i - 1][j], [i - 1, j], [i, j]])
    else:
        arr.append(math.inf)

    #(3) col + 1, row - 1
    if (j + 1 < 8163) and (i - 1 >= 0):
        g = c_anist_cost(i, j, i - 1, j + 1)
        # h = h_cost_euclidean(i - 1, j + 1)
        h = h_cost_manhattan(i - 1, j - 1)

        if (acc_cost[i - 1][j + 1] > (g + h)):
            parent[i - 1][j + 1] = i, j
            acc_cost[i - 1][j + 1] = (g + h)
        arr.append([acc_cost[i - 1][j + 1], [i - 1, j + 1], [i, j]])
    else:
        arr.append(math.inf)

    #(4) col - 1, row
    if (j - 1 >= 0):
        g = c_anist_cost(i, j, i, j - 1)
        # h = h_cost_euclidean(i, j - 1)
        h = h_cost_manhattan(i - 1, j - 1)

        if (acc_cost[i][j - 1] > (g + h)):
            parent[i][j - 1] = i, j
            acc_cost[i][j - 1] = (g + h)
        arr.append([acc_cost[i][j - 1], [i, j - 1], [i, j]])
    else:
        arr.append(math.inf)

    #(5) col + 1, row
    if (j + 1 < 8163):
        g = c_anist_cost(i, j, i, j + 1)
        # h = h_cost_euclidean(i, j + 1)
        h = h_cost_manhattan(i - 1, j - 1)

        if (acc_cost[i][j + 1] > (g + h)):
            parent[i][j + 1] = i, j
            acc_cost[i][j + 1] = (g + h)
        arr.append([acc_cost[i][j + 1], [i, j + 1], [i, j]])
    else:
        arr.append(math.inf)

    #(6) col - 1, row + 1
    if (j - 1 >= 0) and (i + 1 < 10500):
        g = c_anist_cost(i, j, i + 1, j - 1)
        # h = h_cost_euclidean(i + 1, j - 1)
        h = h_cost_manhattan(i - 1, j - 1)

        if (acc_cost[i + 1][j - 1] > (g + h)):
            parent[i + 1][j - 1] = i, j
            acc_cost[i + 1][j - 1] = (g + h)
        arr.append([acc_cost[i + 1][j - 1], [i + 1, j - 1], [i, j]])
    else:
        arr.append(math.inf)

    #(7) col, row + 1
    if (i + 1 < 10500):
        g = c_anist_cost(i, j, i + 1, j)
        # h = h_cost_euclidean(i + 1, j)
        h = h_cost_manhattan(i - 1, j - 1)

        if (acc_cost[i + 1][j] > (g + h)):
            parent[i + 1][j] = i, j
            acc_cost[i + 1][j] = (g + h)
        arr.append([acc_cost[i + 1][j], [i + 1, j], [i, j]])
    else:
        arr.append(math.inf)

    #(8) col + 1, row + 1
    if (j + 1 < 8163) and (i + 1 < 10500):
        g = c_anist_cost(i, j, i + 1, j + 1)
        # h = h_cost_euclidean(i + 1, j + 1)
        h = h_cost_manhattan(i - 1, j - 1)

        if (acc_cost[i + 1][j + 1] > (g + h)):
            parent[i + 1][j + 1] = i, j
            acc_cost[i + 1][j + 1] = (g + h)
        arr.append([acc_cost[i + 1][j + 1], [i + 1, j + 1], [i, j]])
    else:
        arr.append(math.inf)
    

    return arr

In [ ]:
end_pixel

[8344, 5485]

In [ ]:
def h_cost_manhattan(i, j):
    x, y = (i  - end_pixel[0]), (j - end_pixel[1])
    x, y = abs(x), abs(y)
    cst = x + y
    return cst

In [ ]:
def h_cost_euclidean(i, j):
    x, y = (i  - end_pixel[0]), (j - end_pixel[1])
    x, y = x * x, y * y
    cst = math.sqrt(x + y)
    return cst

In [ ]:
# def h_cost_euclidean(i, j):
#     arr = []

#     #(1) col - 1, row - 1
#     if (j - 1 >= 0) and (i - 1 >= 0):
#         x, y = (i - 1) - end_pixel[0], (j - 1) - end_pixel[1]
#         x, y = x * x, y * y
#         arr.append(math.sqrt(x + y))
#     else:
#         arr.append(math.inf)

#     #(2) col, row - 1
#     if (i - 1 >= 0):
#         x, y = ((i - 1) - end_pixel[0]), (j  - end_pixel[1])
#         x, y = x * x, y * y
#         arr.append(math.sqrt(x + y))
#     else:
#         arr.append(math.inf)

#     #(3) col + 1, row - 1
#     if (j + 1 < 8163) and (i - 1 >= 0):
#         x, y = ((i - 1) - end_pixel[0]), ((j + 1)  - end_pixel[1])
#         x, y = x * x, y * y
#         arr.append(math.sqrt(x + y))
#     else:
#         arr.append(math.inf)

#     #(4) col - 1, row
#     if (j - 1 >= 0):
#         x, y = (i - end_pixel[0]), ((j - 1)  - end_pixel[1])
#         x, y = x * x, y * y
#         arr.append(math.sqrt(x + y))
#     else:
#         arr.append(math.inf)

#     #(5) col + 1, row
#     if (j + 1 < 8163):
#         x, y = (i - end_pixel[0]), ((j + 1)  - end_pixel[1])
#         x, y = x * x, y * y
#         arr.append(math.sqrt(x + y))
#     else:
#         arr.append(math.inf)

#     #(6) col - 1, row + 1
#     if (j - 1 >= 0) and (i + 1 < 10500):
#         x, y = ((i + 1) - end_pixel[0]), ((j - 1)  - end_pixel[1])
#         x, y = x * x, y * y
#         arr.append(math.sqrt(x + y))
#     else:
#         arr.append(math.inf)

#     #(7) col, row + 1
#     if (i + 1 < 10500):
#         x, y = ((i + 1) - end_pixel[0]), (j  - end_pixel[1])
#         x, y = x * x, y * y
#         arr.append(math.sqrt(x + y))
#     else:
#         arr.append(math.inf)

#     #(8) col + 1, row + 1
#     if (j + 1 < 8163) and (i + 1 < 10500):
#         x, y = ((i + 1) - end_pixel[0]), ((j + 1)  - end_pixel[1])
#         x, y = x * x, y * y
#         arr.append(math.sqrt(x + y))
#     else:
#         arr.append(math.inf)
    

#     return arr

In [ ]:
dim = mask_data.shape
print(dim)

(10500, 8163)


In [ ]:
start_pixel, end_pixel = [1770, 1442], [8344, 5485]
# start_pixel, end_pixel = [1770, 1442], [1980, 1600]
print(start_pixel, end_pixel)

[1770, 1442] [8344, 5485]


In [ ]:
Q = PriorityQueue()
s_pixel, e_pixel = start_pixel, end_pixel

# mask_copy.shape, dem_data.shape

visited = np.zeros((dim))
acc_cost = np.full((dim), math.inf)

acc_cost[s_pixel[0]][s_pixel[1]] = 0
visited[s_pixel[0]][s_pixel[1]] = 1

parent = np.full((dim), None)
parent[s_pixel[0]][s_pixel[1]] = -1, -1

while (condition_check(s_pixel, e_pixel)):
    
    i, j = s_pixel
    # h-cost(0) - indicates Euclidean distance
    # h-cost(1) - indicates Manhattan distance

    neighbours_cost = get_neigh_cost(i, j)
    for nc in neighbours_cost:
        if nc == math.inf:
            continue
        else:
            Q.put(nc)
    
    bst = Q.get()
    m, n = bst[1][0], bst[1][1]

    while True:
        if visited[m][n] == 0:
            break
        bst = Q.get()
        m, n = bst[1][0], bst[1][1]

    # set-visited
    visited[m][n] = 1
    # path.append([m, n])
    # print(m, n)

    parent[m][n] = bst[2][0], bst[2][1]

    s_pixel = [m, n]

In [ ]:
print(acc_cost[end_pixel[0]][end_pixel[1]])

45398362.40329002


In [ ]:
!rm -rf path.txt

In [ ]:
path = []
pr = parent[end_pixel[0]][end_pixel[1]]

# cnt = 15000
while (pr[0] != -1) and (pr[1] != -1):
    path.append(pr)
    # path.append('-')
    with open('path.txt', 'a') as f:
        f.write(str(pr) + '\n')

    # cnt = cnt - 1
    # if cnt == 0:
    #     break

    pr = parent[pr[0]][pr[1]]

In [ ]:
path_list = []
with open('path.txt', 'r') as f:
    for point in f:
        path_list.append(point)

In [ ]:
len(path_list)

6918

In [ ]:
point = path_list[0].replace('\n', '').split(' ')
pi, pj = int(point[0].split(',')[0].split('(')[1]), int(point[1].split(',')[0].split(')')[0])
print(pi, pj)

8343 5485


In [ ]:
len(path_list)

6918

In [ ]:
path_list.reverse()

In [ ]:
%%time
ordinates_dict = {}


for i in range(len(path_list)):
    point = path_list[i].replace('\n', '').split(' ')
    pi, pj = int(point[0].split(',')[0].split('(')[1]), int(point[1].split(',')[0].split(')')[0])

    ext_i, ext_j = left + (pi * 10), top - (pj * 10)

    # ordinates_dict[(pi, pj)] = extent_matrix[pi][pj]
    ordinates_dict[pi, pj] = [ext_i, ext_j]

CPU times: user 74.4 ms, sys: 2 µs, total: 74.4 ms
Wall time: 74.1 ms


In [ ]:
ordinates_dict
ordinates_list = []
for key, value in ordinates_dict.items():
    ordinates_list.append(value)

In [ ]:
import shapefile
w = shapefile.Writer('/home/shrayank_mistry/Modules/project-mum-pune/routes-shape/a-star-man/shapefiles/test/multipoint')
w.field('name', 'C')

w.multipoint(ordinates_list) 
w.record('multipoint1')

w.close()

In [ ]:
road_lenght = 0
index = 1
for _ in ordinates_list[1:]:
    i, j = ordinates_list[index][0], ordinates_list[index][1]
    x, y = ordinates_list[index - 1][0], ordinates_list[index - 1][1]

    if (x - i == 10.0) and (y - j == -10.0):
        road_lenght += math.sqrt(2 * 100)
    else:
        road_lenght += 10 

In [ ]:
print(f'Current Road-length = {road_lenght / 1000} kms')

Current Road-length = 69.17 kms
